In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_excel('december-2562.xls')

In [3]:
df.head()

,ID,Name,Department,Date,Time,eDate
0,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,01/12/2019,NaN,2019-12-01
1,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,02/12/2019,08:53 19:09,2019-12-02
2,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,03/12/2019,08:17 19:52,2019-12-03
3,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,04/12/2019,07:57 20:52,2019-12-04
4,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,05/12/2019,NaN,2019-12-05


In [4]:
DATETIME_FORMAT = '%d/%m/%Y %H:%M'
def get_check_in_n_out_time(x):
    if not pd.isna(x.Time):
        start, end = x.Time.split()[0], x.Time.split()[-1]
        if start != end:
            start_dt = datetime.strptime(u'{} {}'.format(x.Date, start), DATETIME_FORMAT)
            end_dt = datetime.strptime(u'{} {}'.format(x.Date, end), DATETIME_FORMAT)
            return pd.Series({'start_dt': start_dt, 'end_dt': end_dt})
        else:
            start_dt = datetime.strptime(u'{} {}'.format(x.Date, start), DATETIME_FORMAT)
            # not sure what to do with no check in time
            return pd.Series({'start_dt': start_dt, 'end_dt': None})
    else:
        return pd.Series({'start_dt': None, 'end_dt': None})

In [5]:
def is_late(x, starttime='09:00', endtime='16:30'):
    # minus = early
    # plus = late
    startdt = datetime.strptime(u'{} {}'.format(x.Date, starttime), DATETIME_FORMAT)
    enddt = datetime.strptime(u'{} {}'.format(x.Date, endtime), DATETIME_FORMAT)
    morning = None
    evening = None
    if not pd.isna(x.start_dt):
        if startdt > x.start_dt:
            morning = (startdt - x.start_dt)
            morning = (morning.seconds/60.0) * -1
        else:
            morning = x.start_dt - startdt
            morning = morning.seconds/60.0
    if not pd.isna(x.end_dt):
        if enddt < x.end_dt:
            evening = x.end_dt - enddt
            evening = evening.seconds/60.0
        else:
            evening = enddt - x.end_dt
            evening = (evening.seconds/60.0) * -1
    if morning and evening:
        return pd.Series({'evening_late': evening, 'morning_late': morning})
    return pd.Series({'evening_late': evening, 'morning_late': morning})

In [95]:
def get_no_scan_data(row):
    if pd.isna(row.Time):
        if (row.eDate.weekday_name == 'Sunday'
            or row.eDate.weekday_name == 'Saturday'):
            return u'{}*'.format(row.eDate.date())
        else:
            return str(row.eDate.date())
    else:
        return None

In [96]:
df['no_scan'] = df.apply(get_no_scan_data, axis=1)

In [97]:
df.head()

,ID,Name,Department,Date,Time,eDate,end_dt,start_dt,evening_early,morning_late,on_late,off_early,no_scan
0,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,01/12/2019,NaN,2019-12-01,NaT,NaT,NaN,NaN,NaN,NaN,2019-12-01*
1,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,02/12/2019,08:53 19:09,2019-12-02,2019-12-02 19:09:00,2019-12-02 08:53:00,159.0,-7.0,0.0,0.0,None
2,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,03/12/2019,08:17 19:52,2019-12-03,2019-12-03 19:52:00,2019-12-03 08:17:00,202.0,-43.0,0.0,0.0,None
3,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,04/12/2019,07:57 20:52,2019-12-04,2019-12-04 20:52:00,2019-12-04 07:57:00,262.0,-63.0,0.0,0.0,None
4,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,05/12/2019,NaN,2019-12-05,NaT,NaT,NaN,NaN,NaN,NaN,2019-12-05


In [88]:
df[['end_dt', 'start_dt']] = df.apply(get_check_in_n_out_time, axis=1)

In [7]:
df.head()

,ID,Name,Department,Date,Time,eDate,end_dt,start_dt
0,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,01/12/2019,NaN,2019-12-01,NaT,NaT
1,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,02/12/2019,08:53 19:09,2019-12-02,2019-12-02 19:09:00,2019-12-02 08:53:00
2,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,03/12/2019,08:17 19:52,2019-12-03,2019-12-03 19:52:00,2019-12-03 08:17:00
3,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,04/12/2019,07:57 20:52,2019-12-04,2019-12-04 20:52:00,2019-12-04 07:57:00
4,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,05/12/2019,NaN,2019-12-05,NaT,NaT


In [8]:
df[['evening_early', 'morning_late']] = df.apply(is_late, axis=1)

In [9]:
df.head()

,ID,Name,Department,Date,Time,eDate,end_dt,start_dt,evening_early,morning_late
0,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,01/12/2019,NaN,2019-12-01,NaT,NaT,NaN,NaN
1,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,02/12/2019,08:53 19:09,2019-12-02,2019-12-02 19:09:00,2019-12-02 08:53:00,159.0,-7.0
2,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,03/12/2019,08:17 19:52,2019-12-03,2019-12-03 19:52:00,2019-12-03 08:17:00,202.0,-43.0
3,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,04/12/2019,07:57 20:52,2019-12-04,2019-12-04 20:52:00,2019-12-04 07:57:00,262.0,-63.0
4,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,05/12/2019,NaN,2019-12-05,NaT,NaT,NaN,NaN


In [15]:
def is_evening_early(x):
    if pd.isna(x):
        return None
    else:
        return 1 if x<0 else 0
    
def is_morning_late(x):
    if pd.isna(x):
        return None
    else:
        return 0 if x<0 else 1

In [16]:
df['off_early'] = df.evening_early.apply(is_evening_early)

In [17]:
df['on_late'] = df.morning_late.apply(is_morning_late)

In [18]:
df.head()

,ID,Name,Department,Date,Time,eDate,end_dt,start_dt,evening_early,morning_late,on_late,off_early
0,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,01/12/2019,NaN,2019-12-01,NaT,NaT,NaN,NaN,NaN,NaN
1,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,02/12/2019,08:53 19:09,2019-12-02,2019-12-02 19:09:00,2019-12-02 08:53:00,159.0,-7.0,0.0,0.0
2,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,03/12/2019,08:17 19:52,2019-12-03,2019-12-03 19:52:00,2019-12-03 08:17:00,202.0,-43.0,0.0,0.0
3,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,04/12/2019,07:57 20:52,2019-12-04,2019-12-04 20:52:00,2019-12-04 07:57:00,262.0,-63.0,0.0,0.0
4,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,05/12/2019,NaN,2019-12-05,NaT,NaT,NaN,NaN,NaN,NaN


In [45]:
flt_df = df[(~pd.isna(df['start_dt']) | (~pd.isna(df['end_dt'])))]

In [46]:
flt_df.head()

,ID,Name,Department,Date,Time,eDate,end_dt,start_dt,evening_early,morning_late,on_late,off_early
1,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,02/12/2019,08:53 19:09,2019-12-02,2019-12-02 19:09:00,2019-12-02 08:53:00,159.0,-7.0,0.0,0.0
2,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,03/12/2019,08:17 19:52,2019-12-03,2019-12-03 19:52:00,2019-12-03 08:17:00,202.0,-43.0,0.0,0.0
3,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,04/12/2019,07:57 20:52,2019-12-04,2019-12-04 20:52:00,2019-12-04 07:57:00,262.0,-63.0,0.0,0.0
5,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,06/12/2019,07:34 19:33,2019-12-06,2019-12-06 19:33:00,2019-12-06 07:34:00,183.0,-86.0,0.0,0.0
8,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,09/12/2019,07:52 17:12,2019-12-09,2019-12-09 17:12:00,2019-12-09 07:52:00,42.0,-68.0,0.0,0.0


In [21]:
off_early_df = pd.pivot_table(flt_df, index=['Department', 'Name'], values='off_early', aggfunc='sum')

In [22]:
off_early_df.head()

off_early
Department  Name                         
งานการศึกษา คณุสรณ์ จวนรุ่ง           1.0
            จรูญศรี บุญเรือง          3.0
            จินตนา กลิ่นถือศีล        2.0
            อัญชลี เนียมศิริ          3.0
            อาภรณ์ ครองกิจการ         0.0

In [23]:
on_late_df = pd.pivot_table(flt_df, index=['Name'], values='on_late', aggfunc='sum')

In [24]:
on_late_df.head()

,on_late
Name,
Aijad Ahmad Malik,7.0
กนกวรรณ กิตตินิยม,8.0
กมลรัตน์ โพธิ์ปิน,8.0
กรฤทธิ์ อนันต์ศิลป์,5.0
กรุณา สุทธิวงศ์,1.0


In [25]:
on_late_mins_df = pd.pivot_table(flt_df[flt_df['morning_late']>0], index=['Name'], values='morning_late', aggfunc='sum')

In [26]:
off_early_mins_df = pd.pivot_table(flt_df[flt_df['evening_early']<0], index=['Name'], values='evening_early', aggfunc='sum')

In [125]:
summary_df = off_early_df.join(on_late_df).join(on_late_mins_df).join(off_early_mins_df)

In [126]:
summary_df.head()

off_early  on_late  morning_late  \
Department  Name                                                   
งานการศึกษา คณุสรณ์ จวนรุ่ง           1.0      0.0           NaN   
            จรูญศรี บุญเรือง          3.0      0.0           NaN   
            จินตนา กลิ่นถือศีล        2.0      3.0          19.0   
            อัญชลี เนียมศิริ          3.0      1.0         223.0   
            อาภรณ์ ครองกิจการ         0.0     14.0         775.0   

                                evening_early  
Department  Name                               
งานการศึกษา คณุสรณ์ จวนรุ่ง             -17.0  
            จรูญศรี บุญเรือง            -91.0  
            จินตนา กลิ่นถือศีล          -16.0  
            อัญชลี เนียมศิริ            -35.0  
            อาภรณ์ ครองกิจการ             NaN

In [127]:
flt_df['checkout'] = flt_df[flt_df['off_early']==1].end_dt.dt.strftime('%d/%m/%y %H:%M')

/Users/likit/.virtualenvs/mis2018/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [128]:
flt_df['checkin'] = flt_df[flt_df['on_late']==1].start_dt.dt.strftime('%d/%m/%y %H:%M')

/Users/likit/.virtualenvs/mis2018/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [129]:
flt_df[['checkout', 'checkin']].head()

,checkout,checkin
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
5,NaN,NaN
8,NaN,NaN


In [130]:
off_early_checkouts = flt_df[flt_df['off_early']==1]

In [131]:
off_early_checkout_lists = off_early_checkouts.groupby(['Name', 'Department'])['checkout'].agg({'checkout_datetime': ', '.join})

/Users/likit/.virtualenvs/mis2018/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [132]:
off_early_checkout_lists.head()

,,checkout_datetime
Name,Department,
กรฤทธิ์ อนันต์ศิลป์,ศูนย์เทคนิคการแพทย์ฯ,"04/12/19 16:00, 06/12/19 07:56, 09/12/19 16:01..."
กรุณา สุทธิวงศ์,งานบริการสุขภาพชุมชน,12/12/19 13:12
กฤฏิยากรญ์ ตันติธวัช,งานบริการสุขภาพชุมชน,"01/12/19 12:01, 07/12/19 12:00, 10/12/19 12:00..."
กัญญาภัค ทิพย์เทพ,สถานเวชศาสตร์ชันสูตร,"02/12/19 16:18, 03/12/19 16:20, 04/12/19 16:22..."
กาญจนา กลีบบัว,งานยุทธศาสตร์ และการ,17/12/19 15:06


In [133]:
on_late_checkin = flt_df[flt_df['on_late']==1]

In [134]:
on_late_checkin_lists = on_late_checkin.groupby(['Name', 'Department'])['checkin'].agg({'checkin_datetime': ', '.join})

/Users/likit/.virtualenvs/mis2018/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [135]:
no_scan_df = df[~pd.isna(df.no_scan)]

In [136]:
no_scan_df.head()

,ID,Name,Department,Date,Time,eDate,end_dt,start_dt,evening_early,morning_late,on_late,off_early,no_scan
0,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,01/12/2019,NaN,2019-12-01,NaT,NaT,NaN,NaN,NaN,NaN,2019-12-01*
4,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,05/12/2019,NaN,2019-12-05,NaT,NaT,NaN,NaN,NaN,NaN,2019-12-05
6,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,07/12/2019,NaN,2019-12-07,NaT,NaT,NaN,NaN,NaN,NaN,2019-12-07*
7,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,08/12/2019,NaN,2019-12-08,NaT,NaT,NaN,NaN,NaN,NaN,2019-12-08*
9,27,พจนารถ วานิจจะกุล,ภาควิชาจุลทรรศนศาสตร,10/12/2019,NaN,2019-12-10,NaT,NaT,NaN,NaN,NaN,NaN,2019-12-10


In [137]:
no_scan_summary = no_scan_df.groupby(['Name', 'Department'])['no_scan'].agg({'no_scan_dates': ', '.join, 'no_scan_date_count': 'count'})

/Users/likit/.virtualenvs/mis2018/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [138]:
summary_ = pd.merge(summary_df.reset_index(), off_early_checkout_lists.reset_index()[['Name', 'checkout_datetime']], left_on='Name', right_on='Name', how='outer')

In [139]:
summary_df = pd.merge(summary_, on_late_checkin_lists.reset_index()[['Name', 'checkin_datetime']],
                      left_on='Name', right_on='Name', how='outer')

In [140]:
summary_df.head()

,Department,Name,off_early,on_late,morning_late,evening_early,checkout_datetime,checkin_datetime
0,งานการศึกษา,คณุสรณ์ จวนรุ่ง,1.0,0.0,NaN,-17.0,13/12/19 16:13,NaN
1,งานการศึกษา,จรูญศรี บุญเรือง,3.0,0.0,NaN,-91.0,"11/12/19 16:20, 20/12/19 16:25, 27/12/19 15:14",NaN
2,งานการศึกษา,จินตนา กลิ่นถือศีล,2.0,3.0,19.0,-16.0,"11/12/19 16:26, 27/12/19 16:18","06/12/19 09:05, 20/12/19 09:14, 23/12/19 09:00"
3,งานการศึกษา,อัญชลี เนียมศิริ,3.0,1.0,223.0,-35.0,"03/12/19 16:26, 11/12/19 16:17, 27/12/19 16:12",20/12/19 12:43
4,งานการศึกษา,อาภรณ์ ครองกิจการ,0.0,14.0,775.0,NaN,NaN,"02/12/19 09:25, 03/12/19 09:24, 04/12/19 09:07..."


In [141]:
summary_df = pd.merge(summary_df,
                      no_scan_summary.reset_index()[['Name', 'no_scan_dates', 'no_scan_date_count']],
                      left_on='Name', right_on='Name')

In [142]:
summary_df.head()

,Department,Name,off_early,on_late,morning_late,evening_early,checkout_datetime,checkin_datetime,no_scan_dates,no_scan_date_count
0,งานการศึกษา,คณุสรณ์ จวนรุ่ง,1.0,0.0,NaN,-17.0,13/12/19 16:13,NaN,"2019-12-01*, 2019-12-05, 2019-12-07*, 2019-12-...",13
1,งานการศึกษา,จรูญศรี บุญเรือง,3.0,0.0,NaN,-91.0,"11/12/19 16:20, 20/12/19 16:25, 27/12/19 15:14",NaN,"2019-12-01*, 2019-12-05, 2019-12-07*, 2019-12-...",13
2,งานการศึกษา,จินตนา กลิ่นถือศีล,2.0,3.0,19.0,-16.0,"11/12/19 16:26, 27/12/19 16:18","06/12/19 09:05, 20/12/19 09:14, 23/12/19 09:00","2019-12-01*, 2019-12-02, 2019-12-03, 2019-12-0...",15
3,งานการศึกษา,อัญชลี เนียมศิริ,3.0,1.0,223.0,-35.0,"03/12/19 16:26, 11/12/19 16:17, 27/12/19 16:12",20/12/19 12:43,"2019-12-01*, 2019-12-05, 2019-12-07*, 2019-12-...",13
4,งานการศึกษา,อาภรณ์ ครองกิจการ,0.0,14.0,775.0,NaN,NaN,"02/12/19 09:25, 03/12/19 09:24, 04/12/19 09:07...","2019-12-01*, 2019-12-05, 2019-12-07*, 2019-12-...",16


In [143]:
summary_df.to_excel('december2019_report.xlsx')

In [144]:
!open december2019_report.xlsx